## RQ2: How does bike rental demand vary by hour, season, and holidays?

### Data Processing

In [54]:
import pandas as pd
import numpy as np
from statsmodels.gam.api import GLMGam, BSplines
import statsmodels.api as sm
import statsmodels.gam.api as smg
import altair as alt
alt.data_transformers.disable_max_rows()

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [27]:
# Load the dataset with appropriate encoding
file_path = "SeoulBikeData.csv"
df = pd.read_csv(file_path, encoding='latin1')

# Keep only functioning days
df = df[df['Functioning Day'] == 'Yes'].copy()

# Rename and clean columns
df = df.rename(columns={"Rented Bike Count": "RentedBikeCount"})
df["Holiday"] = df["Holiday"].map({"Holiday": 1, "No Holiday": 0})

# Convert 'Season' to category
df['Season'] = df['Seasons'].astype('category')

# Drop unnecessary columns
df = df[['RentedBikeCount', 'Hour', 'Season', 'Holiday']].copy()

print(df.head(20))
print(df.info())

    RentedBikeCount  Hour  Season  Holiday
0               254     0  Winter        0
1               204     1  Winter        0
2               173     2  Winter        0
3               107     3  Winter        0
4                78     4  Winter        0
5               100     5  Winter        0
6               181     6  Winter        0
7               460     7  Winter        0
8               930     8  Winter        0
9               490     9  Winter        0
10              339    10  Winter        0
11              360    11  Winter        0
12              449    12  Winter        0
13              451    13  Winter        0
14              447    14  Winter        0
15              463    15  Winter        0
16              484    16  Winter        0
17              555    17  Winter        0
18              862    18  Winter        0
19              600    19  Winter        0
<class 'pandas.core.frame.DataFrame'>
Index: 8465 entries, 0 to 8759
Data columns (total 4 column

### Baseline model: GLM

In [49]:
# Create dummies for seasons
season_dummies = pd.get_dummies(df['Season'], drop_first=True).astype(int)

# Combine predictors and add constant
X = pd.concat([df[['Hour', 'Holiday']], season_dummies], axis=1)
X = sm.add_constant(X)

# Response
y = df['RentedBikeCount']

print(X.head())
print(X.dtypes)

   const  Hour  Holiday  Spring  Summer  Winter
0    1.0     0        0       0       0       1
1    1.0     1        0       0       0       1
2    1.0     2        0       0       0       1
3    1.0     3        0       0       0       1
4    1.0     4        0       0       0       1
const      float64
Hour         int64
Holiday      int64
Spring       int64
Summer       int64
Winter       int64
dtype: object


In [29]:
# Fit the GLM using poisson
poisson_model = sm.GLM(y, X, family=sm.families.Poisson())
poisson_result = poisson_model.fit()
print(poisson_result.summary())

# Check for overdispersion
print("Residual deviance:", poisson_result.deviance)
print("Degrees of freedom:", poisson_result.df_resid)


                 Generalized Linear Model Regression Results                  
Dep. Variable:        RentedBikeCount   No. Observations:                 8465
Model:                            GLM   Df Residuals:                     8459
Model Family:                 Poisson   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -1.1845e+06
Date:                Sun, 23 Mar 2025   Deviance:                   2.3018e+06
Time:                        03:46:41   Pearson chi2:                 2.23e+06
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.1177      0.001   5356.043      0.0

In [58]:
# The Residual Deviance is much larger than the degrees of freedom, suggesting overdispersion
# Fit the model using a Negative Binomial distribution to handle the overdispersion
nb_model = sm.GLM(y, X, family=sm.families.NegativeBinomial())
nb_result = nb_model.fit()
print(nb_result.summary())

# Check for overdispersion
print("Residual deviance:", nb_result.deviance)
print("Degrees of freedom:", nb_result.df_resid)

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


                 Generalized Linear Model Regression Results                  
Dep. Variable:        RentedBikeCount   No. Observations:                 8465
Model:                            GLM   Df Residuals:                     8459
Model Family:        NegativeBinomial   Df Model:                            5
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -62401.
Date:                Sun, 23 Mar 2025   Deviance:                       4769.1
Time:                        05:45:47   Pearson chi2:                 3.52e+03
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3571
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.0889      0.029    208.565      0.0

In [42]:
# Calculate percentage change
# Extract coefficients
coefs = nb_result.params

# Exponentiate to get multiplicative effect
percent_change = (np.exp(coefs) - 1) * 100
percent_change = percent_change.round(2)

print(percent_change)

const      43994.94
Hour           5.99
Holiday      -16.14
Spring       -21.04
Summer        11.42
Winter       -74.40
dtype: float64


In [47]:
# Plot the partial effect plot for Hour in GLM
# Define range of hours
hour_vals = np.arange(0, 24)

# Create base DataFrame for prediction
X_glm_hour = pd.DataFrame({
    'const': 1,
    'Hour': hour_vals,
    'Holiday': 0,
    'Spring': 0,
    'Summer': 0,
    'Winter': 0,
})

# Predict using fitted Negative Binomial GLM
y_pred_glm = nb_result.predict(X_glm_hour)

# Combine into plotting DataFrame
df_glm_hour = pd.DataFrame({
    'Hour': hour_vals,
    'Predicted Rentals (GLM)': y_pred_glm
})

alt.Chart(df_glm_hour).mark_line(point=True).encode(
    x=alt.X('Hour:Q', title='Hour of Day'),
    y=alt.Y('Predicted Rentals (GLM):Q', title='Predicted Rental Count'),
).properties(
    title='Partial Effect of Hour on Bike Rentals (GLM)',
    width=500,
    height=300
)

alt.Chart(...)

#### Interpretation  
To investigate how bike rental demand varies by hour, season, and holidays, we fit a Generalized Linear Model (GLM) using a Negative Binomial distribution with a log link. This model was selected over a Poisson GLM due to evidence of overdispersion — the Poisson model’s residual deviance (2.3 million) far exceeded the degrees of freedom (8,459), whereas the Negative Binomial model’s deviance (4,769) aligned closely with the degrees of freedom, indicating a better fit.  
  
Model results show that each additional hour of the day is associated with a 6% increase in expected bike rentals. On holidays, rentals are about 16% lower than on non-holidays. Seasonal variation is also significant: compared to autumn (the reference), rentals are 11% higher in summer, 21% lower in spring, and 74% lower in winter. All coefficients were statistically significant at the 0.05 level.  
  
These results confirm that temporal factors — including time of day, season, and holiday status — strongly influence bike-sharing demand in Seoul. The negative binomial GLM provides a robust framework for modeling these effects while accounting for variability in rental counts.  

### Advanced model: GAM

In [56]:
# Response variable
y = df['RentedBikeCount']

# Base predictors (Holiday is linear, Season is categorical)
X_df = pd.concat([df[['Holiday']], season_dummies], axis=1)
X_df = sm.add_constant(X_df)
print(X_df)

# Create B-Splines for Hour
# Hour is numeric, from 0 to 23
# Use 6 basis functions (knots), cubic splines (degree=3)
bs = BSplines(df[['Hour']], df=[6], degree=[3])

# Fit the GAM
gam_model = GLMGam(y, exog=X_df, smoother=bs, family=sm.families.NegativeBinomial())
gam_result = gam_model.fit()
print(gam_result.summary())

      const  Holiday  Spring  Summer  Winter
0       1.0        0       0       0       1
1       1.0        0       0       0       1
2       1.0        0       0       0       1
3       1.0        0       0       0       1
4       1.0        0       0       0       1
...     ...      ...     ...     ...     ...
8755    1.0        0       0       0       0
8756    1.0        0       0       0       0
8757    1.0        0       0       0       0
8758    1.0        0       0       0       0
8759    1.0        0       0       0       0

[8465 rows x 5 columns]
                 Generalized Linear Model Regression Results                  
Dep. Variable:        RentedBikeCount   No. Observations:                 8465
Model:                         GLMGam   Df Residuals:                  8455.06
Model Family:        NegativeBinomial   Df Model:                         8.94
Link Function:                    Log   Scale:                          1.0000
Method:                         PIRLS   

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "


In [48]:
# Plot the smooth effect of Hour
# Create a grid of hours
hour_grid = pd.DataFrame({'Hour': np.arange(0, 24)})

# Create the same spline basis used during training
bs_hour = smg.BSplines(hour_grid, df=[6], degree=[3])

# Get the design matrix for hour
hour_spline_matrix = bs_hour.transform(hour_grid[['Hour']].values)

# Get the last 5 parameters from the model results
spline_params = gam_result.params[-5:]

# Multiply spline basis by spline coefficients
smooth_hour_effect = hour_spline_matrix @ spline_params

# Prepare data for plotting
plot_df = hour_grid.copy()
plot_df['SmoothedEffect'] = smooth_hour_effect

print(plot_df)

# Plot with Altair
alt.Chart(plot_df).mark_line(point=True).encode(
    x=alt.X('Hour:Q', title='Hour of Day'),
    y=alt.Y('SmoothedEffect:Q', title='Smoothed Effect on log(Rental Count)'),
    tooltip=['Hour', 'SmoothedEffect']
).properties(
    title='Smoothed Effect of Hour on Bike Rentals (GAM)',
    width=500,
    height=300
)

    Hour  SmoothedEffect
0      0        0.000000
1      1       -0.726953
2      2       -1.120653
3      3       -1.245750
4      4       -1.166898
5      5       -0.948747
6      6       -0.655950
7      7       -0.353159
8      8       -0.104505
9      9        0.057086
10    10        0.147054
11    11        0.185005
12    12        0.190541
13    13        0.183266
14    14        0.182781
15    15        0.208692
16    16        0.278033
17    17        0.378005
18    18        0.476557
19    19        0.541316
20    20        0.539911
21    21        0.439971
22    22        0.209124
23    23       -0.185003


alt.Chart(...)

In [41]:
# Compare different models
models = {
    "Poisson": poisson_result,
    "NegBin": nb_result,
    "GAM_NegBin": gam_result
}

performance = {
    name: {
        "Log-Likelihood": model.llf,
        "Residual Deviance": model.deviance
    }
    for name, model in models.items()
}

performance_df = pd.DataFrame(performance).T
print(performance_df)


            Log-Likelihood  Residual Deviance
Poisson      -1.184466e+06       2.301839e+06
NegBin       -6.240116e+04       4.769106e+03
GAM_NegBin   -6.194172e+04       3.850211e+03


In [45]:
# Calculate percentage change
# Extract coefficients
coefs2 = gam_result.params

# Exponentiate to get multiplicative effect
percent_change2 = (np.exp(coefs) - 1) * 100
percent_change2 = percent_change2.round(2)

print(percent_change2)

const      91435.91
Holiday      -14.92
Spring       -21.91
Summer        14.43
Winter       -74.17
Hour_s0      -90.35
Hour_s1      118.15
Hour_s2      -25.03
Hour_s3      188.18
Hour_s4      -16.89
dtype: float64


#### Interpretation:  
To model the nonlinear effect of time of day on bike rental demand, we fit a Generalized Additive Model (GAM) using a Negative Binomial distribution. The model included:  
- A smooth spline term for Hour
- Linear effects for season (with Autumn as the reference category)
- A binary indicator for holidays
- The GAM achieved a better fit than both the Poisson and Negative Binomial GLMs. It showed a lower residual deviance (3,850 vs. 4,769) and a higher log-likelihood (–61,942 vs. –62,401) than the standard Negative Binomial GLM, confirming the benefit of capturing nonlinear temporal structure.
  
The estimated smooth function for Hour is shown above.  
  
The curve reveals a clear nonlinear pattern in rental demand:  
- Rental activity is lowest between midnight and 5 AM, reflecting minimal overnight usage.
- Starting around 6 AM, demand begins rising sharply.
- A broad plateau appears between 8 AM and 5 PM, suggesting steady demand during work hours.
- A second peak emerges between 6 PM and 9 PM, likely due to evening commutes or leisure use.
- After 10 PM, demand begins to drop again.
- The flexible smoothing provided by the GAM captures these subtle variations much more effectively than models assuming a linear hourly trend.
  
Seasonal and holiday effects were also consistent with expectations:  
- Summer increases rental activity by about 14% compared to autumn.
- Spring shows a 22% decrease, while Winter brings a substantial 74% reduction.
- On holidays, rentals are roughly 15% lower than regular days.

  
Overall, the GAM not only fits better statistically but also offers a more interpretable representation of daily and seasonal demand dynamics.

In [69]:
# Create Predicted vs Actual for both GLM and GAM
# Get predictions
y_pred_glm = nb_result.predict()
y_pred_gam = gam_result.predict()

# Combine with actual values
glm_df = pd.DataFrame({
    'Actual': y,
    'Predicted': y_pred_glm,
    'Model': 'GLM'
})

gam_df = pd.DataFrame({
    'Actual': y,
    'Predicted': y_pred_gam,
    'Model': 'GAM'
})

# Set axis limits
x_max = 2000
y_max = 4000

gam_scatter = alt.Chart(gam_df).mark_circle(size=20, opacity=0.3, color="#4C78A8").encode(
    x=alt.X('Predicted:Q', scale=alt.Scale(domain=[0, x_max]), title='Predicted Rental Count'),
    y=alt.Y('Actual:Q', scale=alt.Scale(domain=[0, y_max]), title='Actual Rental Count')
).properties(
    title='GAM',
    width=300,
    height=300
)

gam_line = alt.Chart(pd.DataFrame({'Predicted': [0, x_max], 'Actual': [0, x_max]})).mark_line(
    strokeDash=[4, 4], color='black'
).encode(x='Predicted:Q', y='Actual:Q')

glm_scatter = alt.Chart(glm_df).mark_circle(size=20, opacity=0.3, color="#F58518").encode(
    x=alt.X('Predicted:Q', scale=alt.Scale(domain=[0, x_max]), title='Predicted Rental Count'),
    y=alt.Y('Actual:Q', scale=alt.Scale(domain=[0, y_max]), title='Actual Rental Count')
).properties(
    title='GLM',
    width=300,
    height=300
)

glm_line = alt.Chart(pd.DataFrame({'Predicted': [0, x_max], 'Actual': [0, x_max]})).mark_line(
    strokeDash=[4, 4], color='black'
).encode(x='Predicted:Q', y='Actual:Q')

# Combine the two side by side
chart = (gam_scatter + gam_line) | (glm_scatter + glm_line)

# Add common title
chart = chart.properties(
    title='Predicted vs Actual Rental Count with Diagonal Line'
)

chart


alt.HConcatChart(...)

#### Predicted vs. Actual Plot (GLM vs. GAM)
To evaluate and compare the predictive performance of the Negative Binomial GAM and GLM models, we plotted the predicted rental counts against the actual rental counts for each model. Figure above displays these results side by side, with a reference diagonal line indicating perfect prediction. Observations aligning closely with this line indicate stronger predictive accuracy.
  
The plot for the GAM model (left panel) reveals a relatively tighter clustering of points around the diagonal, particularly in the mid-range of predictions (approximately 500–1500 rentals). This suggests that the GAM is better at capturing the underlying nonlinear relationship between hour of day and rental demand, likely due to the flexibility of the smoothing spline applied to the Hour variable. In contrast, the GLM (right panel) shows a wider spread and greater deviation from the diagonal, especially as predicted values increase. This indicates more systematic under- or over-predictions, particularly in the upper range of rental demand.
  
Overall, the GAM visually demonstrates improved predictive fit compared to the GLM, consistent with the earlier deviance-based comparison and the inclusion of nonlinear temporal effects. This highlights the value of using additive smoothing for time-varying patterns such as hourly demand cycles.